
# Evaualton Idee

- Wir laden ein Dokument
- Für das Dokument bitten wir ein LLM uns eine Frage und eine Antwort zu erzeugen

- Mit den Fragen und Antworten testen wir das LLM - Die Bwertung macht wieder ein LLM.

- Was kann Open Web UI?
- Was für Framworks gibt es?
  - https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/nvidia_metrics/#answer-accuracy

## Datensatz erstellen

In [1]:
# iterate over the docs folder and load all md files


# iterate over the docs folder and load all md files

import os
from pathlib import Path

# Define the docs folder path
docs_folder = Path("docs")

# List to store the content of all markdown files
markdown_documents = []

# Iterate through all files in the docs folder recursively
for md_file in docs_folder.rglob("*.md"):
    try:
        # Read the content of each markdown file
        with open(md_file, 'r', encoding='utf-8') as f:
            content = f.read()
            
        # Store the file information and content
        markdown_documents.append({
            'filename': md_file.name,
            'path': str(md_file),
            'content': content
        })
        
        print(f"Loaded: {md_file}")
        
    except Exception as e:
        print(f"Error loading {md_file}: {e}")

print(f"\nTotal markdown files loaded: {len(markdown_documents)}")

Loaded: docs/PV_Apostille_Haager_Apostille.md
Loaded: docs/Linksammlung_Trennung_mit_Kind.md
Loaded: docs/Erlaubnisfreie_Gewerbe.md
Loaded: docs/Betreuungsverfügung.md
Loaded: docs/Nach_der_Lehre_ohne_Job.md
Loaded: docs/Einreise_aus_EU-Staaten.md
Loaded: docs/List_of_screening_examiniations_and_vaccinations.md
Loaded: docs/Linksammlung_Fahrzeuge.md
Loaded: docs/Gemeinschaftliches_Testament.md
Loaded: docs/PV-Sublage_Kinderbetreuung.md
Loaded: docs/Particular_regulations_according_to_Book_XII_Social_Code_and_Asylum_Seekers_Benefits_Act.md
Loaded: docs/Wer_darf_wählen_Europawahl.md
Loaded: docs/Gesundheitswegweiser.md
Loaded: docs/Banken_und_Zahlungsverkehr.md
Loaded: docs/Vorläufiger_Rechtsschutz_Privatrecht.md
Loaded: docs/Versicherungen.md
Loaded: docs/Flüchtlingskinder.md
Loaded: docs/Sonstige_Reiseversicherungen.md
Loaded: docs/Aufzeichnungen_und_Rechenschaften.md
Loaded: docs/Vorläufiger_Rechtsschutz_öffentliches_Recht.md
Loaded: docs/Pflegeformen.md
Loaded: docs/Der_neue_Strom.md

In [2]:
markdown_documents


[{'filename': 'PV_Apostille_Haager_Apostille.md',
  'path': 'docs/PV_Apostille_Haager_Apostille.md',
  'content': '# PV_Apostille (Haager Apostille)\n\nDamit eine im Inland ausgestellte öffentliche Urkunde auch durch eine ausländische Behörde anerkannt werden kann, wurde für den internationalen Rechtsverkehr die Möglichkeit eines gesonderten Beglaubigungsverfahrens eingeführt (Legalisation). Die Legalisation ist mit einem nicht unerheblichen Zeit- und Kostenaufwand verbunden. Sie ist deshalb durch internationale oder bilaterale Verträge zwischen Staaten, die Vertrauen in das Urkundenwesen des jeweiligen anderen Staates haben, teilweise für entbehrlich erklärt worden.\n\nZu diesen Übereinkommen zählt unter anderem das Übereinkommen zur Befreiung ausländischer öffentlicher Urkunden von der Legalisation (Haager Apostilleübereinkommen). An die Stelle der Legalisation tritt dann als Echtheitsnachweis die Apostille.\n\nDie Apostille wird durch die zuständige innere Behörde des Staates, der d

In [3]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI(
    #base_url = 'http://185.128.119.215:11434/v1',
    base_url='http://localhost:1234/v1',  # Adjust the URL as needed
    api_key="lm-studio"
    # api_key='ollama', # required, but unused
)

class AnswerQuestionPair(BaseModel):
        question: str
        answer: str


class DocAnswerQuestionPair(AnswerQuestionPair):
        source_document: str

def get_question_answer_pair_from_document(doc: str) -> AnswerQuestionPair:
    """
    Generates a meaningful question-answer pair from the document content.
    """
    prompt = """You are an expert in creating question-answer pairs for evaluation purposes.

Given the following text:
---
{document}
---

Task:
1. Carefully analyze the text
2. Create ONE specific, relevant question about the content of the text
3. Formulate a precise, correct answer based on the text

Requirements:
- The question should query important information from the text
- The question should not be too trivial or too complex
- The answer must be directly derivable from the text
- Avoid yes/no questions if possible
- The question should be formulated in natural language

!Important!
- The question and the answer must be in the same language as the given text
"""


    response = client.beta.chat.completions.parse(
        model="gemma-3-12b-it-qat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that extracts precise question-answer pairs from texts."},
            {"role": "user", "content": prompt.format(document=doc)}
        ],
        temperature=0.7,
        response_format=AnswerQuestionPair,
    )
    
    return response.choices[0].message.parsed

In [4]:
get_question_answer_pair_from_document("Club Mate ist lecker und macht wach. Es ist ein Getränk, das Koffein enthält und in Deutschland sehr beliebt ist. Es wird oft in Clubs und Bars serviert und hat einen einzigartigen Geschmack, der viele Menschen anspricht.")

AnswerQuestionPair(question='Wo wird Club Mate oft serviert?', answer='Es wird oft in Clubs und Bars serviert.')

In [ ]:
# iterate over the markdown documents and generate question-answer pairs
from tqdm import tqdm
question_answer_pairs:list[DocAnswerQuestionPair] = []

for doc in tqdm(markdown_documents[:200], desc="Generating Q&A pairs"):
    try:
        qa_pair = get_question_answer_pair_from_document(doc['content'])
        
        # Much cleaner: create DocAnswerQuestionPair directly
        doc_qa_pair = DocAnswerQuestionPair(
            question=qa_pair.question,
            answer=qa_pair.answer,
            source_document=doc['filename']
        )
        question_answer_pairs.append(doc_qa_pair)
        
    except Exception as e:
        print(f"Error generating Q&A for {doc['filename']}: {e}")

Generating Q&A pairs: 100%|██████████| 10/10 [00:53<00:00,  5.31s/it]


In [ ]:
# save the question-answer pairs to a file
import json
from pathlib import Path

# Convert Pydantic models to dictionaries
qa_pairs_dict = [qa.model_dump() for qa in question_answer_pairs]

# Save to JSON file
output_file = Path("question_answer_pairs.json")
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(qa_pairs_dict, f, ensure_ascii=False, indent=2)

print(f"Saved {len(qa_pairs_dict)} question-answer pairs to {output_file}")

Saved 10 question-answer pairs to question_answer_pairs.json
